In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import glob
import numpy as np
import cv2
import copy

# Path
video_files = glob.glob('/content/drive/MyDrive/kaggle/dataset/*.mp4')
frame_count = []

for video_file in copy.deepcopy(video_files):
    cap = cv2.VideoCapture(video_file)

    if int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150:
        video_files.remove(video_file)
        continue

    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

print("Frames:", frame_count)
print("Total number of videos:", len(frame_count))
print('Average frame per video:', np.mean(frame_count))


frames [299, 299, 299, 299, 299, 300, 299, 299, 300, 299, 299, 300, 299, 300, 300, 299, 299, 300, 300, 300]
Total number of videos:  20
Average frame per video: 299.4


In [ ]:
import cv2
import face_recognition
from tqdm.autonotebook import tqdm
import os
import glob

def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success = 1
    while success:
        success, image = vidObj.read()
        if success:
            yield image

!pip3 install face_recognition
!mkdir '/content/drive/MyDrive/kaggle/dataset/face_cropped'

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import numpy as np
import matplotlib.pyplot as plt

def create_face_videos(path_list, out_dir):
    already_present_count = glob.glob(out_dir + '*.mp4')
    print("No of videos already present ", len(already_present_count))

    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, path.split('/')[-1])
        file_exists = glob.glob(out_path)

        if len(file_exists) != 0:
            print("File Already exists: ", out_path)
            continue

        frames = []
        flag = 0
        face_all = []
        frames1 = []

        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (112, 112))

        for idx, frame in enumerate(frame_extract(path)):
            if idx <= 150:
                frames.append(frame)
                if len(frames) == 4:
                    faces = face_recognition.batch_face_locations(frames)
                    for i, face in enumerate(faces):
                        if len(face) != 0:
                            top, right, bottom, left = face[0]
                        try:
                            out.write(cv2.resize(frames[i][top:bottom, left:right, :], (112, 112)))
                        except:
                            pass
                    frames = []

        try:
            del top, right, bottom, left
        except:
            pass

        out.release()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=026417b4d90eee70e2f0872ef63bc4036e158c460233b0fa6e4f3e3190c2ffd2
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
create_face_videos(video_files,'/content/drive/MyDrive/kaggle/dataset/face_cropped')

No of videos already present  0


  0%|          | 0/20 [00:00<?, ?it/s]